In [82]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
import keras
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

In [83]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
set_session(tf.Session(config=config))

In [84]:
ft = fastText.load_model("/home1/zishan/raghav/wiki.hi.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [85]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [86]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [87]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [88]:
def train_dev_sentences(filetrain, filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]
    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)

In [89]:
train_file = '/home1/zishan/raghav/Data/movie_sentiments.txt'
dev_file = '/home1/zishan/raghav/Data/review_sentiments.txt'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(train_file, dev_file, 2)

In [90]:
print(set(train_labels))
print(set(dev_labels))
print(labels2Idx)



{0, 1, 2, 3}
{0, 1, 2, 3}
{'conflict': 0, 'neutral': 1, 'positive': 2, 'negative': 3}


In [91]:
print(len(train_sentences))
print(len(dev_sentences))

2151
10834


In [92]:
train_sentences.extend(dev_sentences)
train_labels.extend(dev_labels)

In [40]:
print(len(train_sentences))

7568


In [75]:
print(labels2Idx)
labels2Idx.pop('cpn',None)
print(labels2Idx)

{'conflict': 0, 'neutral': 1, 'positive': 2, 'negative': 3}
{'conflict': 0, 'neutral': 1, 'positive': 2, 'negative': 3}


In [93]:
from collections import Counter
print(Counter(train_labels))
print(Counter(dev_labels))

Counter({2: 5403, 1: 5050, 3: 1954, 0: 578})
Counter({2: 4580, 1: 4452, 3: 1424, 0: 378})


In [94]:
def compile_model_bilstm(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)
    model_concatenated = Flatten()(lstm_block)
    model_concatenated = Dense(100)(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

In [111]:
def compile_model_bilstm_3cnn(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

In [112]:
no_labels = len(labels2Idx)

In [113]:
model = compile_model_bilstm_3cnn(no_labels)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_13 (Bidirectional (None, 75, 200)      320800      input_13[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_29 (LeakyReLU)      (None, 75, 200)      0           bidirectional_13[0][0]           
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 73, 200)      120200      leaky_re_lu_29[0][0]             
__________________________________________________________________________________________________
conv1d_14 

In [114]:
train_file = '/home1/zishan/raghav/Data/movie_and_review_sentiments.txt'
weights_file ='/home1/zishan/raghav/weights/pretrain_bilstm_3cnn_dropout=0.5.h5'
# log_file = '/home1/zishan/raghav/logs/bilstm.txt'
batch_size = 16
check_for_generator = 2
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [115]:
callback = keras.callbacks.ModelCheckpoint(weights_file, monitor='acc', verbose=0, save_best_only=True, save_weights_only=True)


In [116]:
model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx,tokenize= tokenize),
                        steps_per_epoch= steps_per_epoch, epochs=20, callbacks = [callback])

Epoch 1/20
812/812 [==============================] - 219s 270ms/step - loss: 1.0646 - acc: 0.5062
Epoch 2/20
812/812 [==============================] - 227s 280ms/step - loss: 0.9355 - acc: 0.5840
Epoch 3/20
812/812 [==============================] - 227s 279ms/step - loss: 0.8486 - acc: 0.6330
Epoch 4/20
812/812 [==============================] - 228s 281ms/step - loss: 0.7823 - acc: 0.6620
Epoch 5/20
812/812 [==============================] - 228s 281ms/step - loss: 0.7031 - acc: 0.7004
Epoch 6/20
812/812 [==============================] - 228s 281ms/step - loss: 0.6311 - acc: 0.7391
Epoch 7/20
812/812 [==============================] - 228s 281ms/step - loss: 0.5560 - acc: 0.7683
Epoch 8/20
812/812 [==============================] - 229s 282ms/step - loss: 0.4985 - acc: 0.7993
Epoch 9/20
812/812 [==============================] - 229s 281ms/step - loss: 0.4400 - acc: 0.8229
Epoch 10/20
812/812 [==============================] - 228s 280ms/step - loss: 0.3944 - acc: 0.8430
Epoch 11/